In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas_profiling
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv('/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/train.csv')
test_data = pd.read_csv('/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/test.csv')

In [ ]:
report_train = pandas_profiling.ProfileReport(train_data)
report_train.to_file("report_train.html")

report_train


In [ ]:
report_test = pandas_profiling.ProfileReport(test_data)
report_test.to_file("report_test.html")

report_test


In [ ]:
train_data.info()

In [ ]:
train_data.nunique()


In [ ]:
test_data.nunique()


In [ ]:
merged = pd.concat([train_data,test_data],axis = 0)
merged.dtypes.value_counts()


In [ ]:
cols_with_missing = (col for col in merged.columns 
                                 if merged[col].isnull().any())

for col in cols_with_missing:
    merged[col + '_was_missing'] = merged[col].isnull()

columns = ['Artist Reputation_was_missing', 'Height_was_missing',
       'Width_was_missing', 'Weight_was_missing', 'Material_was_missing',
       'Transport_was_missing', 'Remote Location_was_missing']

for col in columns:
    result = merged[col].astype(int)
    merged[col] = result

In [ ]:
merged['Missing_count'] = merged['Artist Reputation_was_missing'] + merged['Height_was_missing'] + merged['Width_was_missing'] + merged['Weight_was_missing'] + merged['Material_was_missing'] + merged['Transport_was_missing'] + merged['Remote Location_was_missing'] 
merged

In [ ]:
merged = merged.drop(['Cost_was_missing'],axis = 1)

In [ ]:
train_data_mod = merged.iloc[:6500, :]
test_data_mod = merged.iloc[6500:, :].drop(columns = ['Cost'], axis = 1)

In [ ]:
train_data_mod['Missing_count'].value_counts()


In [ ]:
exp_tf = train_data_mod 
#exp_tf_filtered = exp_tf[exp_tf['Missing_count'] < 4] 
exp_tf_filtered = exp_tf

In [ ]:
exp_tf_filtered['Artist Reputation'].fillna(exp_tf_filtered['Artist Reputation'].median(), inplace=True)
exp_tf_filtered['Height'].fillna(exp_tf_filtered['Height'].median(), inplace=True)
exp_tf_filtered['Width'].fillna(exp_tf_filtered['Width'].median(), inplace=True)
exp_tf_filtered['Weight'].fillna(exp_tf_filtered['Weight'].median(), inplace=True)


In [ ]:
exp_tf_filtered['Material'].fillna(exp_tf_filtered['Material'].mode()[0], inplace=True)
exp_tf_filtered['Transport'].fillna(exp_tf_filtered['Transport'].mode()[0], inplace=True)
exp_tf_filtered['Remote Location'].fillna(exp_tf_filtered['Remote Location'].mode()[0], inplace=True)


In [ ]:
exp_tf_filtered.reset_index(drop=True, inplace=True)

In [ ]:
test_data_mod['Artist Reputation'].fillna(exp_tf_filtered['Artist Reputation'].median(), inplace=True)
test_data_mod['Height'].fillna(exp_tf_filtered['Height'].median(), inplace=True)
test_data_mod['Width'].fillna(exp_tf_filtered['Width'].median(), inplace=True)
test_data_mod['Weight'].fillna(exp_tf_filtered['Weight'].median(), inplace=True)


In [ ]:
test_data_mod['Material'].fillna(exp_tf_filtered['Material'].mode()[0], inplace=True)
test_data_mod['Transport'].fillna(exp_tf_filtered['Transport'].mode()[0], inplace=True)
test_data_mod['Remote Location'].fillna(exp_tf_filtered['Remote Location'].mode()[0], inplace=True)

In [ ]:
exp_tf_filtered.info()

In [ ]:
import seaborn as sns
df = exp_tf_filtered.copy()
sns.boxplot(x=df['Artist Reputation'])

In [ ]:
sns.distplot(df['Artist Reputation'])

In [ ]:
sns.boxplot(x=df['Height'])

In [ ]:
sns.distplot(x=df['Height'])

In [ ]:
'''q = df['Height'].quantile(0.999)
df_out1 = df[df['Height']>=q]
df = df[df['Height']<q]

sns.distplot(x=df['Height'])'''

In [ ]:
sns.boxplot(x=exp_tf_filtered['Width'])

In [ ]:
sns.distplot(x=df['Width'])

In [ ]:
'''q2 = df['Width'].quantile(0.999)
df_out2 = df[df['Width']>=q2]
df = df[df['Width']<q2]

sns.distplot(x=df['Width'])'''

In [ ]:
sns.boxplot(x=exp_tf_filtered['Weight'])

In [ ]:
sns.distplot(x=df['Weight'])

In [ ]:
'''q3 = df['Weight'].quantile(0.95)
df_out3 = df[df['Weight']>=q3]
df = df[df['Weight']<q3]

sns.distplot(x=df['Weight'])'''

In [ ]:
sns.boxplot(x=exp_tf_filtered['Price Of Sculpture'])

In [ ]:
sns.distplot(x=exp_tf_filtered['Price Of Sculpture'])

In [ ]:
'''q4 = df['Price Of Sculpture'].quantile(0.99)
df_out4 = df[df['Price Of Sculpture']>=q4]
df = df[df['Price Of Sculpture']<q4]

sns.distplot(x=df['Price Of Sculpture'])'''

In [ ]:
'''dp_removed = pd.concat([df_out1,df_out2, df_out3, df_out4],axis = 0)
dp_removed'''

In [ ]:
df.reset_index(drop=True, inplace=True)


In [ ]:
df = exp_tf_filtered

In [ ]:
target = df.Cost
df.drop('Cost', axis = 1, inplace = True)
cooking_data = pd.concat([df,test_data_mod],axis = 0)


In [ ]:
cooking_data

In [ ]:
cooking_data.info()

In [ ]:
cooking_data.nunique()


In [ ]:
cooking_data['Customer Location']

In [ ]:
cooking_data['zip'] = cooking_data['Customer Location'].map(lambda x:x.split()[-1])
cooking_data['zip'] = cooking_data['zip'].astype('int64')

In [ ]:
cooking_data.drop(['Customer Id', 'Artist Name', 'Customer Location'], axis = 1, inplace = True)

In [ ]:
unencoded_data = cooking_data.copy()

In [ ]:
one_hot_data = unencoded_data[['Material', 'Transport']]
one_hot_data = pd.get_dummies(one_hot_data, drop_first=True, prefix=['Material', 'Transport'])
one_hot_data = one_hot_data.astype('int64')

In [ ]:
bool_data = unencoded_data[['International', 'Express Shipment', 'Installation Included', 'Fragile', 'Remote Location']]

In [ ]:
bool_data.replace(value = [1, 0], to_replace = ['Yes', 'No'], inplace = True)

In [ ]:
label_data = unencoded_data[['Customer Information']]
label_data.replace(value = [1, 0], to_replace = ['Wealthy', 'Working Class'], inplace = True)

In [ ]:
date_data1 = unencoded_data['Scheduled Date']
date_data2 = unencoded_data['Delivery Date']

In [ ]:
date_data1 = pd.to_datetime(date_data1)
date_data2 = pd.to_datetime(date_data2)

In [ ]:
date_data2

In [ ]:
date_data1

In [ ]:
days_diff = date_data2 - date_data1
days_diff = pd.to_numeric(days_diff)
days_diff = days_diff/(24*60*60*1000000000)
days_diff = days_diff.astype(int)

In [ ]:
num_data = unencoded_data.select_dtypes(include = ['int64', 'float64'])
num_data

In [ ]:
cooked_data = pd.concat([num_data, one_hot_data, bool_data, label_data, days_diff],axis = 1)
cooked_data = cooked_data.rename(columns = {0:"Days_diff"}) 

In [ ]:
cooked_data

In [ ]:
cooked_data.drop(['Missing_count'], axis=1, inplace=True)

In [ ]:
cooked_data.info()

In [ ]:
train_prepared =  cooked_data.iloc[:6500, :]
test_prepared = cooked_data.iloc[6500:, :]

In [ ]:
train_prepared

In [ ]:
test_prepared

In [ ]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(train_prepared)

In [ ]:
scaled_train_data = transformer.transform(train_prepared)
scaled_test_data = transformer.transform(test_prepared)

In [ ]:
scaled_train_data = pd.DataFrame(data = scaled_train_data, columns = train_prepared.columns, index = train_prepared.index)
scaled_test_data = pd.DataFrame(data = scaled_test_data, columns = test_prepared.columns, index = test_prepared.index)

In [ ]:
#scaled_train_data = train_prepared
#scaled_test_data = test_prepared

In [ ]:
scaled_train_data.info()

In [ ]:
target = target.abs()

In [ ]:
sns.distplot(target)


In [ ]:
train_labels_mod = np.log1p(target)
#train_labels_mod = target

In [ ]:
sns.distplot(train_labels_mod)


In [ ]:
target_transformer = StandardScaler().fit(target.values.reshape(-1, 1))
scaled_target = target_transformer.transform(target.values.reshape(-1, 1))

In [ ]:
sns.distplot(scaled_target)
#train_labels_mod = scaled_target

In [ ]:
X = scaled_train_data
y = train_labels_mod
data = pd.concat([scaled_train_data, train_labels_mod],axis = 1)


In [ ]:
#get correlations of each features in dataset
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt
model = ExtraTreesRegressor()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

In [ ]:
feat_importances.nlargest(20)

In [ ]:
X.columns

In [ ]:
col_cons = ['Artist Reputation', 'Height', 'Width', 'Weight', 'Price Of Sculpture',
       'Base Shipping Price', 'Material_Brass', 'Material_Bronze',
       'Material_Clay', 'Material_Marble', 'Material_Stone', 'Material_Wood',
       'Transport_Roadways', 'Transport_Waterways', 'International',
       'Express Shipment', 'Installation Included', 'Fragile',
       'Remote Location', 'Customer Information', 'Days_diff']

In [ ]:
drop_col = ['zip']

In [ ]:
scaled_train_data.drop(drop_col, axis = 1, inplace = True)
scaled_test_data.drop(drop_col, axis = 1, inplace = True)

In [ ]:
#scaled_train_data = scaled_train_data[col_cons]
#scaled_test_data = scaled_test_data[col_cons]

In [ ]:
scaled_train_data

In [ ]:
seed = 43
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


linear = LinearRegression(n_jobs = -1)
lasso = Lasso(random_state = seed)
ridge = Ridge(random_state = seed)
elnt = ElasticNet(random_state = seed)
dt = DecisionTreeRegressor(random_state = seed)
knn = KNeighborsRegressor(n_jobs = -1)
rf =  RandomForestRegressor(n_jobs = -1, random_state = seed)
et = ExtraTreesRegressor(n_jobs = -1, random_state = seed)
ab = AdaBoostRegressor(random_state = seed)
gb = GradientBoostingRegressor(random_state = seed)
xgb = XGBRegressor(random_state = seed, n_jobs = -1, learning_rate = 0.05, max_depth=6, n_estimators=1000)
lgb = LGBMRegressor(random_state = seed, n_jobs = -1)

In [ ]:
def train_r2(model):
    model.fit(scaled_train_data,train_labels_mod)
    return model.score(scaled_train_data,train_labels_mod)

In [ ]:
models = [linear, lasso, ridge, elnt, dt, knn, rf, et, ab, gb, xgb, lgb]
training_score = []
for model in models:
    training_score.append(train_r2(model))
    
train_score = pd.DataFrame(data = training_score, columns = ['Training_R2'])
train_score.index = ['LR', 'LSO', 'RIDGE', 'ELNT', 'DT', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'LGB']
train_score = (train_score*100).round(4)

In [ ]:
sns.scatterplot(train_score.index,train_score['Training_R2'])


In [ ]:
def train_test_split_score(model):
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_log_error
    X_train, X_test, Y_train, Y_test = train_test_split(scaled_train_data, train_labels_mod, test_size = 0.2, random_state = seed)
    model.fit(X_train, Y_train)
    prediction = model.predict(X_test)
    mse = mean_squared_error(prediction, Y_test)
    rmse = np.sqrt(mse)
    error_score = [rmse]
    return error_score

In [ ]:
models = [linear,lasso, ridge, elnt, dt, knn, rf, et, ab, gb, xgb, lgb]
train_test_split_rmse = []

for model in models:
    train_test_split_rmse.append(train_test_split_score(model)[0])


In [ ]:
train_test_score = pd.DataFrame(data = train_test_split_rmse, columns = ['Train_Test_RMSE'])
train_test_score.index = ['Linear','LSO', 'RIDGE', 'ELNT', 'DT', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'LGB']
sns.scatterplot(train_test_score.index,train_test_score['Train_Test_RMSE'])


In [ ]:
def cross_validate(model):
    from sklearn.model_selection import cross_val_score
    neg_x_val_score = cross_val_score(model, scaled_train_data, train_labels_mod, cv = 10, n_jobs = -1, scoring = 'neg_mean_squared_error')
    x_val_score = np.round(np.sqrt(-1*neg_x_val_score), 5)
    return x_val_score.mean()

In [ ]:
models = [lasso, ridge, elnt, dt, knn, rf, et, ab, gb, xgb, lgb]
cross_val_scores = []
for model in models:
    cross_val_scores.append(cross_validate(model))

In [ ]:
x_val_score = pd.DataFrame(data = cross_val_scores, columns = ['Cross Validation Scores (RMSE)'])
x_val_score.index = ['LSO', 'RIDGE', 'ELNT', 'DT', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'LGB']
x_val_score = x_val_score.round(5)
x = x_val_score.index
y = x_val_score['Cross Validation Scores (RMSE)']
sns.scatterplot(x,y)

In [ ]:
xgb.fit(scaled_train_data,train_labels_mod)


In [ ]:
preds = xgb.predict(scaled_test_data)

In [ ]:
#preds = target_transformer.inverse_transform(preds)
preds = np.expm1(preds)

In [ ]:
Id = test_data['Customer Id'].values

In [ ]:
d = {'Customer Id': Id , 'Cost': preds}
submission = pd.DataFrame(data=d)

In [ ]:
submission.to_csv('XGB18.csv', index = False)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_train_data,train_labels_mod,test_size=.2, random_state=42)


In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
# "Learn" the mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE is {:.2f}".format(mae_baseline))

In [ ]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [ ]:
params['eval_metric'] = "mae"


In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=9999,
    evals=[(dtest, "Test")],
    early_stopping_rounds=50
)

In [ ]:
print("Best MAE: {:.5f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

In [ ]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=9999,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=50
)
cv_results

In [ ]:
cv_results['test-mae-mean'].min()


In [ ]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# iteration I found that the optimal value was in the
# following ranges.
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(3,10)
    for min_child_weight in range(0,7)
]

In [ ]:
'''# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=9999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=50
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))'''

In [ ]:
params['max_depth'] = 6
params['min_child_weight'] = 5

In [ ]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(3,11)]
    for colsample in [i/10. for i in range(3,11)]
]

In [ ]:
'''min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=9999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=50
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))'''

In [ ]:
params['subsample'] = 1
params['colsample_bytree'] = 1

In [ ]:
'''%time
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .03, .01, .005, .001]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=9999,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=50
          )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))'''

In [ ]:
params['eta'] = .05


In [ ]:
params

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=9999,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
print("Best MAE: {:.5f} in {} rounds".format(model.best_score, model.best_iteration+1))

In [ ]:
'''num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

In [ ]:
mean_absolute_error(best_model.predict(dtest), y_test)
